<a href="https://colab.research.google.com/github/Nik8x/Deep_writing_generating_text/blob/master/Deep_Writing_with_sentence_word_prediction_lstm_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import regex as re

import nltk
from nltk.draw.dispersion import dispersion_plot
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
nltk.download('averaged_perceptron_tagger')
from nltk.util import ngrams
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding

from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

import urllib

import os

# Any results you write to the current directory are saved as output.
# from google.colab import drive
# drive.mount('/content/gdrive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
Using TensorFlow backend.


In [2]:
text = urllib.request.urlopen('http://www.textfiles.com/stories/3gables.txt').read().decode('utf8')
text = text.replace('\n', ' ').replace('\r', '').replace("\'", "").replace('\w+', '')[348:]
text[0:1000]

'   I dont think that any of my adventures with Mr. Sherlock Holmes opened quite so abruptly, or so dramatically, as that which I associate with The Three Gables. I had not seen Holmes for some days and had no idea of the new channel into which his activities had been directed. He was in a chatty mood that morning, however, and had just settled me into the well-worn low armchair on one side of the fire, while he had curled down with his pipe in his mouth upon the opposite chair, when our visitor arrived. If I had said that a mad bull had arrived it would give a clearer impression of what occurred.   The door had flown open and a huge negro had burst into the room. He would have been a comic figure if he had not been terrific, for he was dressed in a very loud gray check suit with a flowing salmon-coloured tie. His broad face and flattened nose were thrust forward, as his sullen dark eyes, with a smouldering gleam of malice in them, turned from one of us to the other.   "Which of you ge

In [0]:
# I will first convert all the words to numbers, then normalize them, then make a dataframe with two columns, 
# one with sentences(one word, then two words, and so on), the other with the exact next word

In [0]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
encoded = tokenizer.texts_to_sequences([text])[0]

In [5]:
tokenizer.sequences_to_texts([encoded])

['i dont think that any of my adventures with mr sherlock holmes opened quite so abruptly or so dramatically as that which i associate with the three gables i had not seen holmes for some days and had no idea of the new channel into which his activities had been directed he was in a chatty mood that morning however and had just settled me into the well worn low armchair on one side of the fire while he had curled down with his pipe in his mouth upon the opposite chair when our visitor arrived if i had said that a mad bull had arrived it would give a clearer impression of what occurred the door had flown open and a huge negro had burst into the room he would have been a comic figure if he had not been terrific for he was dressed in a very loud gray check suit with a flowing salmon coloured tie his broad face and flattened nose were thrust forward as his sullen dark eyes with a smouldering gleam of malice in them turned from one of us to the other which of you genlmen is masser holmes he

In [6]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 1442


In [0]:
# x = pd.DataFrame(pd.Series([encoded[:i] for i in range(len(encoded))]))
# x['label'] = pd.DataFrame([i for i in encoded])
# x.head()

In [8]:
listy = []

for i in range(len(encoded)):
  listy.append(encoded[:i])
  
len(listy)

6110

In [9]:
listy[1:10]

[[2],
 [2, 64],
 [2, 64, 69],
 [2, 64, 69, 9],
 [2, 64, 69, 9, 102],
 [2, 64, 69, 9, 102, 7],
 [2, 64, 69, 9, 102, 7, 26],
 [2, 64, 69, 9, 102, 7, 26, 562],
 [2, 64, 69, 9, 102, 7, 26, 562, 19]]

In [0]:
# np.array([np.array(xi) for xi in listy[1:]]) # creating array retaining the shape

In [11]:
length = len(sorted(listy[1:],key = len, reverse = True)[0])   # creating array by adding 0 to make all rows equal
X = np.array([xi + [0] * (length - len(xi)) for xi in listy[1:]])
X.shape  # creating X array

(6109, 6109)

In [12]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [13]:
X[0:100]

array([[ 0.00000000e+00, -7.81536947e+01, -5.52584835e+01, ...,
        -2.21657396e-02, -1.80967688e-02, -1.27953004e-02],
       [ 0.00000000e+00,  1.27953004e-02, -5.52584835e+01, ...,
        -2.21657396e-02, -1.80967688e-02, -1.27953004e-02],
       [ 0.00000000e+00,  1.27953004e-02,  1.80967688e-02, ...,
        -2.21657396e-02, -1.80967688e-02, -1.27953004e-02],
       ...,
       [ 0.00000000e+00,  1.27953004e-02,  1.80967688e-02, ...,
        -2.21657396e-02, -1.80967688e-02, -1.27953004e-02],
       [ 0.00000000e+00,  1.27953004e-02,  1.80967688e-02, ...,
        -2.21657396e-02, -1.80967688e-02, -1.27953004e-02],
       [ 0.00000000e+00,  1.27953004e-02,  1.80967688e-02, ...,
        -2.21657396e-02, -1.80967688e-02, -1.27953004e-02]])

In [14]:
y = [i for i in encoded][1:] # creating y array
len(y)

6109

In [0]:
# one hot encode outputs
y = to_categorical(y, num_classes = vocab_size)

In [16]:
y[0:100]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
X.shape, y.shape

((6109, 6109), (6109, 1442))

In [18]:
# LSTMs accept input in the form of (number_of_sequences, length_of_sequence, number_of_features)
X_lstm = np.reshape(X, (X.shape[0], X.shape[1], 1))
X_lstm.shape

(6109, 6109, 1)

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(50, input_shape = (X_lstm.shape[1], X_lstm.shape[2]), return_sequences = True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(50, return_sequences = True))
model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.LSTM(500, return_sequences = True))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.LSTM(500, return_sequences = True))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.LSTM(2500, return_sequences = True))
# model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(50))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 6109, 50)          10400     
_________________________________________________________________
dropout (Dropout)            (None, 6109, 50)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 6109, 50)          20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 6109, 50)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_2 (Dropout)  

In [20]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.80.46.26:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8675534048844622726)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8104626418201717315)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11794380128605886940)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1,

In [21]:
tpu_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_input (InputLayer)      (None, 6109, 1)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 6109, 50)          10400     
_________________________________________________________________
dropout (Dropout)            (None, 6109, 50)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 6109, 50)          20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 6109, 50)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
__________

In [0]:
#early_stopping_monitor = EarlyStopping(patience = 4)
tpu_model.fit(X_lstm, y, epochs = 100, batch_size = 100)#, callbacks = [early_stopping_monitor])

Epoch 1/100
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(12,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(12, 6109, 1), dtype=tf.float32, name='lstm_input_10'), TensorSpec(shape=(12, 1442), dtype=tf.float32, name='dense_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for lstm_input
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:KerasCrossShard: <tensorflow.python.keras.optimizers.Adam object at 0x7f212ce6fbe0> []
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 11.09944772720337 secs
INFO:tensorflow:Setting weights on TPU model.
INFO:tensorflow:CPU -> TPU lr: 0.0010000000474974513 {0.001}
INFO:tensorflow:CPU -> T

In [0]:
# Create the plot
plt.plot(tpu_model.history.history['acc'], 'r', tpu_model.history.history['loss'], 'b')
plt.xlabel('Epochs')
plt.ylabel('Validation score')
plt.legend()
plt.show()

In [0]:
cpu_model = tpu_model.sync_to_cpu()

In [0]:
cpu_model.save_weights('Deep_Writing_with_sentence-word_prediction-tpu.h5')

In [0]:
cpu_model.load_weights('Deep_Writing_with_sentence-word_prediction-tpu.h5')

In [0]:
X[99]

In [0]:
X_lstm[0:10].shape

In [0]:
pred = cpu_model.predict(X_lstm[10:20])

In [0]:
print(pred.shape)
pred

In [0]:
for i in range(len(pred)):
  print(tokenizer.index_word[np.argmax(pred[i])])